## Classic Contrarian Patterns

In [59]:
import backtesting
import tradingUtils as ut
from tradingUtils import OurStrategy
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)
import numpy as np

Loading BokehJS ...

In [60]:
tsla_data = ut.tsla_data

# Marabozu Pattern

Raro de ocorrer

In [61]:
def marubozu_pattern_signal(data):
    result = pd.Series(0, index=data.index)

    close = data['Adj Close'].values
    open_ = data['Open'].values
    high = data['High'].values
    low = data['Low'].values

    for i in range(len(data)):
        
        # Padrão bullish (Marubozu de alta)
        if (
            close[i] > open_[i] and 
            high[i] == close[i] and 
            low[i] == open_[i]
        ):
            result.iloc[i] = 1  # Buy signal

        # Padrão bearish (Marubozu de baixa)
        elif (
            close[i] < open_[i] and 
            high[i] == open_[i] and 
            low[i] == close[i]
        ):
            result.iloc[i] = -1  # Sell signal

    return result



In [62]:
# Gerando os sinais que dão match com a política
policy = marubozu_pattern_signal(tsla_data)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                    0.196574
Equity Final [$]                 10423.954779
Equity Peak [$]                  10677.634779
Return [%]                           4.239548
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                    0.294265
Volatility (Ann.) [%]                1.683178
Sharpe Ratio                         0.174827
Sortino Ratio                        0.471345
Calmar Ratio                         0.112883
Max. Drawdown [%]                   -2.606805
Avg. Drawdown [%]                   -1.977101
Max. Drawdown Duration     4388 days 00:00:00
Avg. Drawdown Duration     2196 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                       3.570243
Worst Trade [%]                      0.646496
Avg. Trade [%]                    

## Three Candles


In [63]:
# Política de três candles como mesmo sinal
def three_candles_policy(data, body):
    result = pd.Series(0, index=data.index)

    # Começamos da terceira linha
    for i in range(2, len(data)):
        close = data['Adj Close'].values
        open_ = data['Open'].values

        # Checamos as condições de bullish
        if (
            (close[i] - open_[i] > body) and
            (close[i - 1] - open_[i - 1] > body) and
            (close[i - 2] - open_[i - 2] > body) and
            close[i] > close[i - 1] and
            close[i - 1] > close[i - 2]
        ):
            result.iloc[i] = 1  # Buy signal
        
        # Checamos as condições de bearish
        elif (
            (close[i] - open_[i] > body) and
            (close[i - 1] - open_[i - 1] > body) and
            (close[i - 2] - open_[i - 2] > body) and
            close[i] < close[i - 1] and
            close[i - 1] < close[i - 2]
        ):
            result.iloc[i] = -1  # Sell signal
    
    return result

In [64]:
# Gerando os sinais que dão match com a política
policy = three_candles_policy(tsla_data, 0.1)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                     11.2328
Equity Final [$]                   9742.21079
Equity Peak [$]                  12510.461014
Return [%]                          -2.577892
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                   -0.184651
Volatility (Ann.) [%]               18.155531
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -41.505972
Avg. Drawdown [%]                  -26.516596
Max. Drawdown Duration     3072 days 00:00:00
Avg. Drawdown Duration     1020 days 00:00:00
# Trades                                  107
Win Rate [%]                        39.252336
Best Trade [%]                      42.505807
Worst Trade [%]                    -14.029062
Avg. Trade [%]                    

## Tasuki Pattern

In [65]:
def tasuki_pattern_policy(data):
    result = pd.Series(0, index=data.index)

    # Pegamos a coluna close e open  
    close = data['Adj Close'].values
    open_ = data['Open'].values
    
    # Começamos da terceira linah
    for i in range(2, len(data)):

        # tasuki de alta
        if (
            (close[i - 2] > open_[i - 2]) and 
            (close[i - 1] > open_[i - 1]) and 
            (open_[i - 1] > close[i - 2]) and  # Checa o gap
            (open_[i] < close[i - 1]) and  # Abre no corpo do segudno
            (close[i] < open_[i]) and  # é de baixa
            (close[i] > close[i - 2])  
        ):
            result.iloc[i] = 1  

        # Tasuki de baixa
        elif (
            (close[i - 2] < open_[i - 2]) and  
            (close[i - 1] < open_[i - 1]) and  
            (open_[i - 1] < close[i - 2]) and 
            (open_[i] > close[i - 1]) and  
            (close[i] > open_[i]) and  
            (close[i] < close[i - 2]) 
        ):
            result.iloc[i] = -1  

    return result

In [66]:
# Gerando os sinais que dão match com a política
policy = tasuki_pattern_policy(tsla_data)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                    2.864364
Equity Final [$]                  8536.160868
Equity Peak [$]                  11610.764615
Return [%]                         -14.638391
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                   -1.113801
Volatility (Ann.) [%]                5.907481
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -29.15745
Avg. Drawdown [%]                  -11.388859
Max. Drawdown Duration     3773 days 00:00:00
Avg. Drawdown Duration     1452 days 00:00:00
# Trades                                   51
Win Rate [%]                        41.176471
Best Trade [%]                       8.228611
Worst Trade [%]                     -8.850762
Avg. Trade [%]                    

## Three Methods Pattern

Não teve nenhum sinal

## Hikkake Pattern

In [67]:
def hikkake_pattern_policy(data):
    result = pd.Series(0, index=data.index)

    close = data['Adj Close'].values
    open_ = data['Open'].values
    high = data['High'].values
    low = data['Low'].values

    for i in range(4, len(data)):  # Start from the 5th data point
        # Bullish Hikkake pattern
        if (
            close[i] > high[i - 3] and
            close[i] > close[i - 4] and
            low[i - 1] < open_[i] and
            close[i - 1] < close[i] and
            high[i - 1] <= high[i - 3] and
            low[i - 2] < open_[i] and
            close[i - 2] < close[i] and
            high[i - 2] <= high[i - 3] and
            high[i - 3] < high[i - 4] and
            low[i - 3] > low[i - 4] and
            close[i - 4] > open_[i - 4]
        ):
            result.iloc[i] = 1  # Buy signal for the next day

        # Bearish Hikkake pattern
        elif (
            close[i] < low[i - 3] and
            close[i] < close[i - 4] and
            high[i - 1] > open_[i] and
            close[i - 1] > close[i] and
            low[i - 1] >= low[i - 3] and
            high[i - 2] > open_[i] and
            close[i - 2] > close[i] and
            low[i - 2] >= low[i - 3] and
            low[i - 3] > low[i - 4] and
            high[i - 3] < high[i - 4] and
            close[i - 4] < open_[i - 4]
        ):
            result.iloc[i] = -1  # Sell signal for the next day

    return result


In [68]:
# Gerando os sinais que dão match com a política
policy = hikkake_pattern_policy(tsla_data)
ut.exit_points(tsla_data, policy, 5, 1)

tsla_data["Signal"] = 0
tsla_data.loc[policy.index, "Signal"] = policy

# Rodando o backtest
bt = Backtest(tsla_data, OurStrategy, cash=10000)
stats = bt.run()

# Exibindo o resultado
bt.plot()
print(stats)

C:\Users\kauan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Start                     2010-06-29 00:00:00
End                       2024-08-21 00:00:00
Duration                   5167 days 00:00:00
Exposure Time [%]                    0.926706
Equity Final [$]                 11261.056065
Equity Peak [$]                  11489.806065
Return [%]                          12.610561
Buy & Hold Return [%]            13918.624358
Return (Ann.) [%]                    0.844004
Volatility (Ann.) [%]                4.392969
Sharpe Ratio                         0.192126
Sortino Ratio                        0.330619
Calmar Ratio                         0.097514
Max. Drawdown [%]                   -8.655173
Avg. Drawdown [%]                   -5.111383
Max. Drawdown Duration     4575 days 00:00:00
Avg. Drawdown Duration     1208 days 00:00:00
# Trades                                    9
Win Rate [%]                        55.555556
Best Trade [%]                       5.358688
Worst Trade [%]                      -4.42278
Avg. Trade [%]                    